In [33]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras import Model
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [34]:
report = pd.read_csv('../input/reports1.csv')#Cleaned the dataset in excel
report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   allTags           498 non-null    object 
 1   createdAt         500 non-null    object 
 2   data              500 non-null    object 
 3   day               67 non-null     float64
 4   desc              115 non-null    object 
 5   downloads         500 non-null    int64  
 6   group             136 non-null    object 
 7   imageLinks        35 non-null     object 
 8   link              500 non-null    object 
 9   month             67 non-null     float64
 10  mostRelevantTags  497 non-null    object 
 11  objectId          500 non-null    object 
 12  pages             500 non-null    int64  
 13  pdf               500 non-null    object 
 14  poster            499 non-null    object 
 15  publisherKey      364 non-null    float64
 16  summary           478 non-null    object 
 1

In [35]:
report.isnull().sum()

allTags               2
createdAt             0
data                  0
day                 433
desc                385
downloads             0
group               364
imageLinks          465
link                  0
month               433
mostRelevantTags      3
objectId              0
pages                 0
pdf                   0
poster                1
publisherKey        136
summary              22
title                25
titleTags            26
toc                 328
updatedAt             0
user                364
views                 0
year                106
dtype: int64

In [36]:
report=report.fillna(0)
report.head()

,allTags,createdAt,data,day,desc,downloads,group,imageLinks,link,month,...,poster,publisherKey,summary,title,titleTags,toc,updatedAt,user,views,year
0,"Brazil, 1, , China, 10, , France, 5, ...",2019-12-16T20:15:56.060Z,manufacturing nike global boston consulting g...,0.0,0,0,0,0,https://media-publications.bcg.com/france/True...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,1.0,0,0,0,0,2020-01-18T15:22:15.580Z,0,5,0.0
1,0,2020-02-15T08:10:06.515Z,Open Source Software Market,0.0,0,0,"{'__type': 'Pointer', 'className': 'MagicSearc...",0,https://icfoss.in/doc/ICFOSS_economic-impact-f...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,0.0,https://api.ydtheyash.com/parse/files/bitgenie...,0,0,0,2020-02-15T16:18:27.680Z,"{'__type': 'Pointer', 'className': '_User', 'o...",2,0.0
2,"India, 18, , Global, 1, , Banking, 1, ...",2020-02-28T10:26:34.660Z,india global technology o B2B customer beha...,0.0,0,0,"{'__type': 'Pointer', 'className': 'MagicSearc...",0,http://www.ijsrp.org/research-paper-0518/ijsrp...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,0.0,https://api.ydtheyash.com/parse/files/bitgenie...,O,O',0,2020-02-28T10:27:52.127Z,"{'__type': 'Pointer', 'className': '_User', 'o...",1,2018.0
3,"Brazil, 1, , China, 1, , Mexico, 2, ,...",2019-12-16T20:55:02.521Z,food tyson foods iot global jefferies globalco...,0.0,0,0,0,0,https://s22.q4cdn.com/104708849/files/doc_pres...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,2.0,https://api.ydtheyash.com/parse/files/bitgenie...,Jefferies GlobalConsumer Conference,"Jefferies', 'GlobalConsumer', 'Conference'",0,2020-01-18T15:22:16.805Z,0,0,2016.0
4,"Germany, 1, , Mali, 1, , New Zealand, ...",2019-12-16T20:16:39.958Z,finance global reshaping nycha support functio...,0.0,0,0,0,0,https://www1.nyc.gov/assets/nycha/downloads/pd...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,1.0,https://api.ydtheyash.com/parse/files/bitgenie...,Reshaping NYCHA support functions,"Reshaping', 'NYCHA', 'support', 'functions'",https://api.ydtheyash.com/parse/files/bitgenie...,2020-01-18T15:22:18.036Z,0,9,2012.0


In [37]:
report['data']= report['data'].astype(str)
report['mostRelevantTags']= report['mostRelevantTags'].astype(str)

In [38]:
g = report.groupby('data')['views'].count()
top_data = g.sort_values(ascending=False)

g = report.groupby('mostRelevantTags')['views'].count()
top_relTags = g.sort_values(ascending=False)

In [39]:
top_r = report.join(top_data, rsuffix='_r', how='inner', on='data')
top_r = top_r.join(top_relTags, rsuffix='_r', how='inner', on='mostRelevantTags')
top_r

,allTags,createdAt,data,day,desc,downloads,group,imageLinks,link,month,...,summary,title,titleTags,toc,updatedAt,user,views,year,views_r,views_r
0,"Brazil, 1, , China, 10, , France, 5, ...",2019-12-16T20:15:56.060Z,manufacturing nike global boston consulting g...,0.0,0,0,0,0,https://media-publications.bcg.com/france/True...,0.0,...,0,0,0,0,2020-01-18T15:22:15.580Z,0,5,0.0,1,1
1,0,2020-02-15T08:10:06.515Z,Open Source Software Market,0.0,0,0,"{'__type': 'Pointer', 'className': 'MagicSearc...",0,https://icfoss.in/doc/ICFOSS_economic-impact-f...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,0,0,0,2020-02-15T16:18:27.680Z,"{'__type': 'Pointer', 'className': '_User', 'o...",2,0.0,1,3
40,"Regis, 1, Companies",2019-12-16T22:11:05.424Z,2020/2021 rpm base residual auction results j...,0.0,0,0,0,0,https://www.pjm.com/~/media/markets-ops/rpm/rp...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,2020/2021 RPM Base Residu...,"2020/2021', 'RPM', 'Base', 'Residual', 'Auctio...",0,2020-01-18T15:22:44.041Z,0,0,2017.0,1,3
330,0,2019-12-31T20:42:11.372Z,boston consulting group bcg,0.0,0,0,0,0,https://bcgcertification.org/wp-content/upload...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,0,0,0,2020-01-18T15:27:45.208Z,0,2,2019.0,1,3
2,"India, 18, , Global, 1, , Banking, 1, ...",2020-02-28T10:26:34.660Z,india global technology o B2B customer beha...,0.0,0,0,"{'__type': 'Pointer', 'className': 'MagicSearc...",0,http://www.ijsrp.org/research-paper-0518/ijsrp...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,O,O',0,2020-02-28T10:27:52.127Z,"{'__type': 'Pointer', 'className': '_User', 'o...",1,2018.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,"Afghanistan, 1, , Albania, 2, , Algeria...",2019-12-17T06:50:40.198Z,europe cloud education ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿...,0.0,0,0,0,0,https://ukznextendedlearning.com/wp-content/up...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½,ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½',https://api.ydtheyash.com/parse/files/bitgenie...,2020-01-18T15:29:27.411Z,0,1,0.0,1,1
494,"Argentina, 2, , Australia, 16, , Belgiu...",2019-12-17T06:52:27.946Z,artificial intelligence united states educatio...,0.0,0,0,0,0,https://www.oecd.org/education/2030/E2030%20Po...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,The fuTure of educaTion and skillsEducation ...,"The', 'fuTure', 'educaTion', 'skillsEducation'...",0,2020-01-18T15:29:28.632Z,0,0,0.0,1,1
495,"Benin, 1, , Botswana, 1, , Burkina, 1,...",2019-12-17T06:52:58.477Z,tanzania education tanzania unesco,0.0,0,0,0,0,https://poledakar.iiep.unesco.org/sites/defaul...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,Tanzania,Tanzania',0,2020-01-18T15:29:29.764Z,0,3,0.0,1,1
496,"Algeria, 4, , Australia, 1, , Austria, ...",2019-12-17T07:03:48.219Z,qatar regis education eurid-unesco world repor...,0.0,0,0,0,https://api.ydtheyash.com/parse/files/bitgenie...,https://www.icann.org/en/system/files/files/eu...,0.0,...,https://api.ydtheyash.com/parse/files/bitgenie...,EURid-UNESCO World report on Internationalise...,"EURid-UNESCO', 'World', 'report', 'Internation...",https://api.ydtheyash.com/parse/files/bitgenie...,2020-01-18T15:29:30.993Z,0,0,2012.0,1,1


In [40]:
data_enc = LabelEncoder()
report['data'] = data_enc.fit_transform(report['data'].values)
n_data = report['data'].nunique()
reltag_enc = LabelEncoder()
report['mostRelevantTags'] = reltag_enc.fit_transform(report['mostRelevantTags'].values)
n_reltags = report['mostRelevantTags'].nunique()
report['views'] = report['views'].values.astype(np.int64)
min_views = min(report['views'])
max_views = max(report['views'])
n_data, n_reltags, min_views, max_views

(473, 395, 0, 211)

In [41]:
X = report[['data','mostRelevantTags']].values
y = report['views'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((400, 2), (100, 2), (400,), (100,))

In [42]:
n_factors = 50
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [43]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2
def RecommenderV1(n_data, n_reltags, n_factors):
    data = Input(shape=(1,))
    d = Embedding(n_data, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(data)
    d = Reshape((n_factors,))(d)
    
    mostRelevantTags = Input(shape=(1,))
    r = Embedding(n_reltags, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(mostRelevantTags)
    r = Reshape((n_factors,))(r)
    
    x = Dot(axes=1)([d,r])
    model = Model(inputs=[data, mostRelevantTags], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [44]:
model = RecommenderV1(n_data, n_reltags, n_factors)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 1, 50)        23650       input_9[0][0]                    
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 1, 50)        19750       input_10[0][0]                   
____________________________________________________________________________________________

In [45]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,verbose=1, validation_data=(X_test_array, y_test))

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 400 samples, validate on 100 samples
Epoch 1/5
400/400 [==============================] - 0s 341us/step - loss: 50.2691 - val_loss: 446.2371
Epoch 2/5
400/400 [==============================] - 0s 38us/step - loss: 50.2318 - val_loss: 446.2363
Epoch 3/5
400/400 [==============================] - 0s 39us/step - loss: 50.1932 - val_loss: 446.2353
Epoch 4/5
400/400 [==============================] - 0s 34us/step - loss: 50.1567 - val_loss: 446.2343
Epoch 5/5
400/400 [==============================] - 0s 35us/step - loss: 50.1239 - val_loss: 446.2334


In [46]:
from keras.layers import Add, Activation, Lambda
class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, x):
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x
def RecommenderV2(n_data, n_reltags, n_factors, min_views, max_views):
    data = Input(shape=(1,))
    d = EmbeddingLayer(n_data, n_factors)(data)
    db = EmbeddingLayer(n_data, 1)(data)
    
    mostRelevantTags = Input(shape=(1,))
    r = EmbeddingLayer(n_reltags, n_factors)(mostRelevantTags)
    rb = EmbeddingLayer(n_reltags, 1)(mostRelevantTags)
    x = Dot(axes=1)([d,r])
    x = Add()([x, db, rb])
    x = Activation('tanh')(x)
    x = Lambda(lambda x: x * (max_views - min_views) + min_views)(x)
    model = Model(inputs=[data,mostRelevantTags], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [47]:
model = RecommenderV2(n_data, n_reltags, n_factors, min_views, max_views)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 1, 50)        23650       input_11[0][0]                   
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 1, 50)        19750       input_12[0][0]                   
____________________________________________________________________________________________

In [48]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,verbose=1, validation_data=(X_test_array, y_test))

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 400 samples, validate on 100 samples
Epoch 1/5
400/400 [==============================] - 0s 500us/step - loss: 527.8367 - val_loss: 774.6845
Epoch 2/5
400/400 [==============================] - 0s 37us/step - loss: 395.1291 - val_loss: 753.4243
Epoch 3/5
400/400 [==============================] - 0s 37us/step - loss: 299.3441 - val_loss: 736.6385
Epoch 4/5
400/400 [==============================] - 0s 37us/step - loss: 228.6687 - val_loss: 722.2638
Epoch 5/5
400/400 [==============================] - 0s 38us/step - loss: 176.3327 - val_loss: 708.6723


In [49]:
from keras.layers import Concatenate, Dense, Dropout
def RecommenderNet(n_data, n_reltags, n_factors, min_views, max_views):
    data = Input(shape=(1,))
    d = EmbeddingLayer(n_data, n_factors)(data)
    
    mostRelevantTags = Input(shape=(1,))
    r = EmbeddingLayer(n_reltags, n_factors)(mostRelevantTags)
    
    x = Concatenate()([d,r])
    x = Dropout(0.05)(x)
    
    x = Dense(10, kernel_initializer='he_normal')(x)
    x = Activation('tanh')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_views - min_views) + min_views)(x)
    model = Model(inputs=[data,mostRelevantTags], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [50]:
model = RecommenderNet(n_data, n_reltags, n_factors, min_views, max_views)
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 1, 50)        23650       input_13[0][0]                   
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 1, 50)        19750       input_14[0][0]                   
____________________________________________________________________________________________

In [51]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=10,verbose=1, validation_data=(X_test_array, y_test))

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 400 samples, validate on 100 samples
Epoch 1/10
400/400 [==============================] - 0s 731us/step - loss: 11148.1092 - val_loss: 10800.0193
Epoch 2/10
400/400 [==============================] - 0s 45us/step - loss: 10432.1832 - val_loss: 10288.0989
Epoch 3/10
400/400 [==============================] - 0s 40us/step - loss: 9604.3031 - val_loss: 9785.4786
Epoch 4/10
400/400 [==============================] - 0s 40us/step - loss: 8819.2346 - val_loss: 9292.6285
Epoch 5/10
400/400 [==============================] - 0s 41us/step - loss: 7976.2400 - val_loss: 8818.2176
Epoch 6/10
400/400 [==============================] - 0s 40us/step - loss: 7349.6748 - val_loss: 8352.4179
Epoch 7/10
400/400 [==============================] - 0s 39us/step - loss: 6737.8833 - val_loss: 7904.2035
Epoch 8/10
400/400 [==============================] - 0s 40us/step - loss: 6210.9459 - val_loss: 7469.0691
Epoch 9/10
400/400 [==============================] - 0s 43us/step - loss: 5524.6666 - val_lo

In [ ]:
###############################THE END###################################